In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
df = pd.read_csv('testing_hour_prices2.csv', sep=';',decimal = ',')


In [3]:
df

,Hour,electricity_price
0,1,0.04
1,2,2.73
2,3,4.40
3,4,2.68
4,5,2.63
...,...,...
244,245,9.59
245,246,8.31
246,247,0.34
247,248,1.74


In [4]:
class EV:
    def __init__(self, name, charge_init):
        self.name = name
        self.charge_init = charge_init
        self.charging_speed = 10
        self.price_memory = [[] for i in range(24)]
        self.average_price_memory = []
        
    
    def fill_memory(self, current_hour, electricity_price):
        '''
        Fills the memory of agents with the previous prices
        
        SHOULD BE DONE ON SUPERCLASS LEVEL TO SAVE DATA AND COMPUTATIONS
        
        '''
        
        self.price_memory[(current_hour %24) - 1].append(electricity_price)
        
    def average_memory(self):
        '''
        From self.price_memory creates avarage prices for a 24h cycle
        
        Could be expanded to a 7*24h cycle if wanted
        
        '''
        self.average_price_memory = [round(np.mean(self.price_memory[i]),2) for i in range(len(self.price_memory))]
    
    
    def choose_cheapest_hours(self,starting_time,ending_time,charge_needed):
        '''This function will tell you the most economic (cheap) way of getting to a full charge within the time window, if possible
           The start and end time are ticks of 1 hour atm
           Charge needed still abstract/dimensionless, the amount of energy the car needs e.g. full or like 75% idc
           
           
           Function use:
           input starting and ending time of charge 
           function outputs cheapest predicted hours (ticks count of hour)
           hours can be set to charging? = true using this
        '''
        if starting_time%24 < ending_time%24:
            total_time_window = self.average_price_memory[starting_time%24:ending_time%24] #e.g. charging from 1AM to 3PM is from 1:00 - 3:00
        else:
            total_time_window = self.average_price_memory.copy()
            del total_time_window[ending_time%24:starting_time%24]
            print('time_window:')
            print(total_time_window)
        hours_needed = math.ceil(charge_needed/self.charging_speed)
        if hours_needed > (abs(ending_time-starting_time)):
            print('total time is insufficient to charge to full. Charging commencing immediately')
            return starting_time

        timewindow_copy = total_time_window.copy()
        timewindow_copy.sort()
        cheapest_values = timewindow_copy[:hours_needed]
        cheapest_starting_hours = [total_time_window.index(i) + starting_time for i in cheapest_values]

        print('the cheapest hour to start are hours {} with a total value of {}'.format(cheapest_starting_hours,cheapest_values))
        
        
        

In [5]:
EV1 = EV('test',20)

In [6]:
for i in range(200):
    EV1.fill_memory(df['Hour'][i], round(df['electricity_price'][i],2))

In [7]:
EV1.price_memory

[[0.04, 8.25, 1.69, 0.96, 6.51, 8.15, 9.66, 9.32, 9.24],
 [2.73, 1.22, 7.41, 3.6, 9.52, 7.71, 0.95, 1.08, 0.95],
 [4.4, 2.56, 7.02, 8.15, 5.69, 8.17, 7.13, 2.15, 7.13],
 [2.68, 8.07, 9.04, 8.02, 0.93, 4.05, 6.67, 3.69, 8.43],
 [2.63, 1.35, 4.42, 2.94, 6.21, 4.03, 3.65, 0.27, 5.23],
 [0.62, 0.41, 9.1, 4.45, 6.8, 6.97, 9.3, 2.11, 4.28],
 [8.56, 4.03, 1.95, 0.58, 6.36, 8.22, 4.95, 1.24, 5.59],
 [3.24, 5.28, 1.24, 3.83, 3.67, 4.04, 9.17, 1.31, 1.93],
 [2.98, 9.23, 3.8, 0.75, 6.77, 9.59, 7.33, 3.83],
 [9.36, 6.97, 7.01, 2.46, 5.6, 2.84, 4.77, 4.61],
 [7.89, 4.9, 1.32, 4.72, 0.5, 6.9, 2.78, 2.14],
 [0.69, 4.0, 3.25, 3.8, 8.33, 3.16, 3.88, 1.69],
 [8.58, 0.79, 9.47, 4.32, 8.94, 9.82, 8.02, 3.13],
 [1.32, 3.33, 7.52, 2.71, 2.25, 1.79, 1.94, 3.31],
 [7.81, 4.03, 1.78, 5.23, 9.34, 7.53, 4.2, 3.65],
 [0.03, 8.08, 3.99, 0.27, 6.45, 8.84, 0.57, 5.98],
 [1.58, 3.79, 5.38, 5.16, 1.43, 1.47, 2.24, 1.7],
 [0.73, 7.26, 7.42, 8.93, 4.41, 2.82, 6.66, 7.31],
 [1.2, 2.13, 6.34, 6.82, 9.69, 6.12, 0.6, 0.88],

In [8]:
EV1.average_memory()

In [9]:
EV1.average_price_memory

[5.98,
 3.91,
 5.82,
 5.73,
 3.41,
 4.89,
 4.61,
 3.75,
 5.54,
 5.45,
 3.89,
 3.6,
 6.63,
 3.02,
 5.45,
 4.28,
 2.84,
 5.69,
 4.22,
 6.35,
 4.89,
 6.62,
 5.22,
 5.83]

In [10]:
EV1.choose_cheapest_hours(21,7,32)

time_window:
[5.98, 3.91, 5.82, 5.73, 3.41, 4.89, 4.61, 6.62, 5.22, 5.83]
the cheapest hour to start are hours [25, 22, 27, 26] with a total value of [3.41, 3.91, 4.61, 4.89]


In [11]:
a = [5.98,3.91,5.82,5.73,3.41,4.89,4.61,3.75,
     5.54,5.45,3.89,3.6,6.63,3.02,5.45,4.28,2.84,5.69,4.22,6.35,4.89,6.62,5.22,5.83]

In [12]:
def choose_cheapest_hours2(charging_speed,charge_needed,average_prices):
        timeslots = []
        hours_needed = math.ceil(charge_needed/charging_speed)
        for i in range(len(average_prices)+1):
            hour_subset = []
            for j in range(3):
                hour_subset.append(a[(i+j)%24])
            timeslots.append(round(sum(hour_subset),3))
        
        cheapest_starting_hour = np.argmin(timeslots) 
        value = timeslots[np.argmin(timeslots)]
        print('the cheapest hour to start is hour {} with a total value of {}'.format(cheapest_starting_hour,value))
        return timeslots
           
        

In [13]:
choose_cheapest_hours(5,15,a)


NameError: name 'choose_cheapest_hours' is not defined

In [ ]:
def choose_cheapest_hours(starting_time,ending_time,charging_speed,charge_needed,average_daily_prices):
        '''This function will tell you the most economic (cheap) way of getting to a full charge within the time window, if possible
        The function does still assume a time that goes from 0 to 23 (1 day), but that can be amended by %izing the start and end time'''
        if starting_time%24 < ending_time%24:
            total_time_window = average_daily_prices[starting_time%24:ending_time%24] #e.g. charging from 1AM to 3PM is from 1:00 - 3:00
        else:
            total_time_window = average_daily_prices.copy()
            del total_time_window[ending_time%24:starting_time%24]
            print('time_window:')
        print(total_time_window)
        hours_needed = math.ceil(charge_needed/charging_speed)
        if hours_needed > (abs(ending_time-starting_time)):
            print('total time is insufficient to charge to full. Charging commencing immediately')
            return starting_time
        
        #timeslots_prices = []                                                This function assumed all needed to be in one line after each other
        #for i in range(len(total_time_window)+1):                            However that is not the case
            #hour_subset = []          
            #for j in range(3):
                #hour_subset.append(total_time_window[(i+j)%len(total_time_window)])
            #timeslots_prices.append(round(sum(hour_subset),3))
        timewindow_copy = total_time_window.copy()
        timewindow_copy.sort()
        cheapest_values = timewindow_copy[:hours_needed]
        cheapest_starting_hours = [total_time_window.index(i) + starting_time for i in cheapest_values]
        
            
            
        
        
        print('the cheapest hour to start are hours {} with a total value of {}'.format(cheapest_starting_hours,cheapest_values))
        
           

In [ ]:
choose_cheapest_hours(15,8,5,15,a)

In [ ]:
def choose_cheapest_hours(starting_time,ending_time,charging_speed,charge_needed,average_daily_prices):
        '''This function will tell you the most economic (cheap) way of getting to a full charge within the time window, if possible
        The function does still assume a time that goes from 0 to 23 (1 day), but that can be amended by %izing the start and end time'''
        if starting_time%24 < ending_time%24:
            total_time_window = average_daily_prices[starting_time%24:ending_time%24] #e.g. charging from 1AM to 3PM is from 1:00 - 3:00
        else:
            total_time_window = average_daily_prices.copy()
            del total_time_window[ending_time%24:starting_time%24]
            print('time_window:')
        
        hours_needed = math.ceil(charge_needed/charging_speed)
        if hours_needed > (abs(ending_time-starting_time)):
            print('total time is insufficient to charge to full. Charging commencing immediately')
            return starting_time

        timewindow_copy = total_time_window.copy()
        timewindow_copy.sort()
        cheapest_values = timewindow_copy[:hours_needed]
        cheapest_starting_hours = [total_time_window.index(i) + starting_time for i in cheapest_values]

        print('the cheapest hour to start are hours {} with a total value of {}'.format(cheapest_starting_hours,cheapest_values))